In [2]:
from kafka import KafkaConsumer
import redis
from vars import topics, sql_conf, tables
import json
import pyodbc

In [3]:
consumer = KafkaConsumer(
    topics["rtt_topic"],
    bootstrap_servers=['172.31.70.21:9092'],
    auto_offset_reset="earliest",
    enable_auto_commit=True,
    group_id=topics["rtt_topic"] + '_group',
    value_deserializer=lambda x: json.loads(x.decode('utf-8'))
)
conn = pyodbc.connect(
    f'DRIVER={sql_conf["driver"]};SERVER=' + sql_conf["server"] + ';DATABASE=' + sql_conf["database"] + \
    ';UID=' + sql_conf["username"] + ';PWD=' + sql_conf["password"])
cursor = conn.cursor()

In [4]:
r = redis.Redis(host="localhost", port=6379, db=0)

In [5]:
def query_ingest():
    query_itemid = "select b.NAMEALIAS from RETAILTRANSACTIONTABLE c " \
                       " inner join RETAILTRANSACTIONSALESTRANS d on " \
                       "c.TRANSACTIONID = d.TRANSACTIONID " \
                       "inner join INVENTTABLE b on " \
                       "b.ITEMID = d.ITEMID where c.TRANSACTIONID = 'Template-000013-338'"
    cursor.execute(query_itemid)
    value = cursor.fetchall()
    r.append('Template-000013-338', str(value))
    return value

In [6]:
list_items = ["TRANSACTIONID", "STORE", "TRANSTIME", "PAYMENTAMOUNT", "CREATEDDATETIME", "CUSTACCOUNT"]

In [7]:
def data_fetch(dic):
    list_items = ["TRANSACTIONID", "STORE", "TRANSTIME", "PAYMENTAMOUNT", "CREATEDDATETIME", "CUSTACCOUNT"]
    var = {k: v for k, v in dic.items() if k in [val for val in list_items]}
    return var

In [9]:
for msg in consumer:
    msg = msg.value
    data = data_fetch(msg["payload"]["after"])
    data["NAMEALIAS"] = query_ingest()
    print(data)

AttributeError: 'NoneType' object has no attribute 'decode'

In [1]:
import pandas as pd

In [10]:
data = pd.read_csv("test.csv")

In [13]:
data.columns

Index(['sum_RETAIL_TRANSACTION_TABLE.PAYMENTAMOUNT', 'sum_sum_RETAIL_TRANSACTION_TABLE.PAYMENTAMOUNT'], dtype='object')

In [14]:
data["sum_RETAIL_TRANSACTION_TABLE.PAYMENTAMOUNT"].sum()

40406599

In [16]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45 entries, 0 to 44
Data columns (total 2 columns):
 #   Column                                          Non-Null Count  Dtype
---  ------                                          --------------  -----
 0   sum_RETAIL_TRANSACTION_TABLE.PAYMENTAMOUNT      45 non-null     int64
 1   sum_sum_RETAIL_TRANSACTION_TABLE.PAYMENTAMOUNT  45 non-null     int64
dtypes: int64(2)
memory usage: 848.0 bytes


In [17]:
data2 = pd.read_csv("test2.csv")

In [18]:
data2

,sum_RETAIL_TRANSACTION_TABLE.PAYMENTAMOUNT,sum_sum_RETAIL_TRANSACTION_TABLE.PAYMENTAMOUNT
0,4198140,4198140
1,3836700,3836700
2,3473200,3473200
3,3171000,3171000
4,2866900,2866900
5,2471500,2471500
6,2447190,2447190
7,2422100,2422100
8,2226000,2226000
9,2058000,2058000


In [19]:
data2["sum_RETAIL_TRANSACTION_TABLE.PAYMENTAMOUNT"].sum()

40406599

In [22]:
temp = {k: v for (k, v) in range(1, 10) }
temp

TypeError: cannot unpack non-iterable int object

In [23]:
d = {1: None, 2: "Hello"}

In [24]:
d

{1: None, 2: 'Hello'}

In [38]:
if 'Hello' in d.values():
    print('Yes')
else:
    print('No')

Yes
